In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('kaggle_movie_train.csv')
data.head(5)

,id,text,genre
0,0,"eady dead, maybe even wishing he was. INT. 2ND...",thriller
1,2,"t, summa cum laude and all. And I'm about to l...",comedy
2,3,"up Come, I have a surprise.... She takes him ...",drama
3,4,ded by the two detectives. INT. JEFF'S APARTME...,thriller
4,5,"nd dismounts, just as the other children reach...",drama


In [3]:
data.shape

(22579, 3)

In [4]:
genre_list=data['genre'].value_counts().index.sort_values(ascending=True)

In [5]:
map_list={k:i for i,k in enumerate(genre_list,0)}

In [6]:
map_list

{'action': 0,
 'adventure': 1,
 'comedy': 2,
 'drama': 3,
 'horror': 4,
 'other': 5,
 'romance': 6,
 'sci-fi': 7,
 'thriller': 8}

In [7]:
data['genre_enc']=data['genre'].map(map_list)

In [8]:
data.head(5)

,id,text,genre,genre_enc
0,0,"eady dead, maybe even wishing he was. INT. 2ND...",thriller,8
1,2,"t, summa cum laude and all. And I'm about to l...",comedy,2
2,3,"up Come, I have a surprise.... She takes him ...",drama,3
3,4,ded by the two detectives. INT. JEFF'S APARTME...,thriller,8
4,5,"nd dismounts, just as the other children reach...",drama,3


In [9]:
data.drop(['id','genre'],axis=1,inplace=True)

In [10]:
data.head(5)

,text,genre_enc
0,"eady dead, maybe even wishing he was. INT. 2ND...",8
1,"t, summa cum laude and all. And I'm about to l...",2
2,"up Come, I have a surprise.... She takes him ...",3
3,ded by the two detectives. INT. JEFF'S APARTME...,8
4,"nd dismounts, just as the other children reach...",3


In [11]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\H'Anu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
corpus=[]
ps=PorterStemmer()
for i in range(0,data.shape[0]):
    dialog=re.sub('[^A-Za-z]',' ',data['text'][i])
    dialog=dialog.lower()
    words=dialog.split()
    dialog_words=[word for word in words if word not in set(stopwords.words('english'))]
    words=[ps.stem(word) for word in dialog_words]
    dialog=' '.join(words)
    corpus.append(dialog)

In [14]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000,ngram_range=(1,2))
X=cv.fit_transform(corpus).toarray()
y=data['genre_enc'].values
pickle.dump(cv, open('cv-transform.pkl','wb'))

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [16]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB(alpha=0.1)
model.fit(X_train,y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [17]:
pickle.dump(model,open('movie-genre-mnb.pkl','wb'))